In [1]:
import pandas as pd

In [2]:
# speads outputted by M3_Predictions.py
submission = pd.read_csv('mydata/mens/original_spreads.csv')
submission.head()

,ID,Pred,TeamID_x,TeamID_y
0,2021_1101_1104,13.818721,1104,1101
1,2021_1101_1116,7.573496,1116,1101
2,2021_1101_1124,15.001942,1124,1101
3,2021_1101_1140,5.961992,1140,1101
4,2021_1101_1155,5.496988,1155,1101


In [3]:
# Switches Pred to the (-1 * Pred) if TeamID_x is not the lower ID as Pred needs to be the spread for the lower ID team
def switch_teams(row):
    idlist = row['ID'].split('_')
    spread = row['Pred']
    if row['TeamID_x'] != int(idlist[1]):
        row['Pred'] = -1 * spread
    row['TeamID_x'] = int(idlist[1])
    row['TeamID_y'] = int(idlist[2])
    return row

submission = submission.apply(switch_teams, axis = 1)

In [4]:
# File for names of teams
team_names = pd.read_csv('ncaam-march-mania-2021/MTeams.csv')[['TeamID', 'TeamName']]
team_names.head()

,TeamID,TeamName
0,1101,Abilene Chr
1,1102,Air Force
2,1103,Akron
3,1104,Alabama
4,1105,Alabama A&M


In [5]:
# Seeds for 2021
seeds = pd.read_csv('ncaam-march-mania-2021/MNCAATourneySeeds.csv').query('Season == 2021')[['TeamID', 'Seed']]
seeds.head()

,TeamID,Seed
2286,1276,W01
2287,1104,W02
2288,1400,W03
2289,1199,W04
2290,1160,W05


In [6]:
# join names with seeds
team_names_seeds = pd.merge(team_names, seeds, on = 'TeamID')
team_names_seeds.head()

,TeamID,TeamName,Seed
0,1101,Abilene Chr,W14
1,1104,Alabama,W02
2,1111,Appalachian St,X16a
3,1116,Arkansas,Z03
4,1124,Baylor,Z01


In [7]:
# Tournament slots for 2021
slots = pd.read_csv('ncaam-march-mania-2021/MNCAATourneySlots.csv').query('Season == 2021')[['Slot', 'StrongSeed', 'WeakSeed']]
slots.head()

,Slot,StrongSeed,WeakSeed
2251,R1W1,W01,W16
2252,R1W2,W02,W15
2253,R1W3,W03,W14
2254,R1W4,W04,W13
2255,R1W5,W05,W12


In [8]:
# join slots with names and seeds
team_slots = pd.merge(slots, team_names_seeds, left_on = ['StrongSeed'], right_on = ['Seed'])
team_slots = pd.merge(team_slots, team_names_seeds, left_on = ['WeakSeed'], right_on = ['Seed']).drop(columns = ['Slot', 'StrongSeed', 'WeakSeed', 'Seed_x', 'Seed_y'])
team_slots.head()

,TeamID_x,TeamName_x,TeamID_y,TeamName_y
0,1104,Alabama,1233,Iona
1,1400,Texas,1101,Abilene Chr
2,1199,Florida St,1422,UNC Greensboro
3,1160,Colorado,1207,Georgetown
4,1163,Connecticut,1268,Maryland


In [9]:
# function to switch slots so the lower id is in first slot
def switch_slots(row):
    if row['TeamID_x'] > row['TeamID_y']:
        team_id = row['TeamID_x']
        name = row['TeamName_x']
        row['TeamID_x'] = row['TeamID_y']
        row['TeamID_y'] = team_id
        row['TeamName_x'] = row['TeamName_y']
        row['TeamName_y'] = name
    return row

team_slots = team_slots.apply(switch_slots, axis = 1)
team_slots.head()

,TeamID_x,TeamName_x,TeamID_y,TeamName_y
0,1104,Alabama,1233,Iona
1,1101,Abilene Chr,1400,Texas
2,1199,Florida St,1422,UNC Greensboro
3,1160,Colorado,1207,Georgetown
4,1163,Connecticut,1268,Maryland


In [10]:
submission['Pred'] = submission['Pred'] * -1

In [11]:
# merge submission with team slots to look at Round 1 probabilities
submission_r1 = pd.merge(submission, team_slots, on = ['TeamID_x', 'TeamID_y'])
submission_r1

,ID,Pred,TeamID_x,TeamID_y,TeamName_x,TeamName_y
0,2021_1101_1400,12.303665,1101,1400,Abilene Chr,Texas
1,2021_1155_1353,1.666064,1155,1353,Clemson,Rutgers
2,2021_1156_1222,16.209415,1156,1222,Cleveland St,Houston
3,2021_1116_1159,-6.823175,1116,1159,Arkansas,Colgate
4,2021_1179_1455,1.704834,1179,1455,Drake,Wichita St
5,2021_1180_1228,15.705873,1180,1228,Drexel,Illinois
6,2021_1186_1242,9.512806,1186,1242,E Washington,Kansas
7,2021_1160_1207,-5.519946,1160,1207,Colorado,Georgetown
8,2021_1210_1260,2.489495,1210,1260,Georgia Tech,Loyola-Chicago
9,2021_1213_1234,13.188303,1213,1234,Grand Canyon,Iowa


In [12]:
ids = list(submission_r1['ID'])

#The lines from Draftkings, inputted manually, in order of IDs in submission_r1
vegas_spreads_x = [
    9, # ABC vs Texas
    1.5, # clemson rutgers
    20, # cleveland state houston
    -8.5, # arkansas colgate
    -1.5, # drake wichita
    22.5, # drexel illinois
    10.5, # e washington kansas
    -6.5, # colorado georgetown
    5.5, # gt loyola
    14.5, # gcu iowa
    -25.5, # baylor hartford
    -16.5, # alabama iona
    7.5, # liberty oklahoma st
    -3, # uconn maryland
    -2, # msu ucla
    1, # mizz oklahoma
    13, # morehead wvu
    -3, # app st norfolk st
    -1.5, # unc wisconsin
    7.5, # north texas purdue
    7.5, # ohio virginia
    -16, # ohio st oral roberts
    8.5, # oregon st tenn
    -7.5, # creighton uscsb
    -1.5, # lsu sb
    -3, # sdsu syracuse
    0, # mt st marys texas southern
    -10.5, # florda st uncg
    -4, # texas tech utah state
    -5, # oregon vcu
    0, # Florida va tech
    -6.5 # Nova Winthrop
] # the spread for TeamID_x (lower team id)

# make datframe of these odds
spread = pd.DataFrame({'ID': ids, 'Spread_x': vegas_spreads_x})
spread.head()

,ID,Spread_x
0,2021_1101_1400,9.0
1,2021_1155_1353,1.5
2,2021_1156_1222,20.0
3,2021_1116_1159,-8.5
4,2021_1179_1455,-1.5


In [13]:
# merge vegas odds with submission probabilities for round 1
submission_r1 = pd.merge(submission_r1, spread, on = 'ID')

# Calculate absolute difference between my probabilities and Vegas'
submission_r1['AbsDiff'] = abs(submission_r1['Spread_x'] - submission_r1['Pred'])

# print submission_r1 ordering by the difference in win probability
submission_r1.sort_values('AbsDiff')

,ID,Pred,TeamID_x,TeamID_y,TeamName_x,TeamName_y,Spread_x,AbsDiff
12,2021_1251_1329,7.371238,1251,1329,Liberty,Oklahoma St,7.5,0.128762
1,2021_1155_1353,1.666064,1155,1353,Clemson,Rutgers,1.5,0.166064
11,2021_1104_1233,-16.755543,1104,1233,Alabama,Iona,-16.5,0.255543
21,2021_1326_1331,-16.311216,1326,1331,Ohio St,Oral Roberts,-16.0,0.311216
24,2021_1261_1382,-1.947825,1261,1382,LSU,St Bonaventure,-1.5,0.447825
28,2021_1403_1429,-4.822719,1403,1429,Texas Tech,Utah St,-4.0,0.822719
7,2021_1160_1207,-5.519946,1160,1207,Colorado,Georgetown,-6.5,0.980054
6,2021_1186_1242,9.512806,1186,1242,E Washington,Kansas,10.5,0.987194
13,2021_1163_1268,-4.064403,1163,1268,Connecticut,Maryland,-3.0,1.064403
9,2021_1213_1234,13.188303,1213,1234,Grand Canyon,Iowa,14.5,1.311697


In [14]:
# adjusts all spreads for a given team by the given amount of points
# positive adj means team is undervalued by model
# negative adj means team is overvalued by model
def team_adjustment(row, team_id, adj):
    idlist = row['ID'].split('_')
    
    # increase spread for team by adj amount
    if team_id == int(idlist[1]):
        return row['Pred'] + adj
    elif team_id == int(idlist[2]):
        return row['Pred'] - adj
    else:
        return row['Pred']
    
# look at Vegas odds, and see which teams the model undervalues that make sense (injuries, COVID, other reasons)
# and adjust their win probabilities for all games manually using function above

# Teams overvalued by my model that I agree with Vegas on
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 1458, 3), axis = 1) # Wisconsin
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 1455, 4), axis = 1) # Wichita
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 1199, 1), axis = 1) # Florida St
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 1317, 2), axis = 1) # North Texas
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 1417, 4), axis = 1) # UCLA
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 1361, 1.7), axis = 1) # SDSU
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 1429, 2), axis = 1) # Utah St


# Teams undervalued by my model that I agree with Vegas on
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 1332, -4), axis = 1) # Oregon
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 1433, -1), axis = 1) # VCU
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 1397, -2), axis = 1) # Tenn
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 1393, -1.7), axis = 1) # Syracuse
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 1393, -1), axis = 1) # UConn
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 1155, -1), axis = 1) # Clemson

# Injuries
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 1437, 6.5), axis = 1) # Villanova
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 1276, 2), axis = 1) # Michigan

# COVID
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 1438, 2), axis = 1) # Virginia
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 1242, 1), axis = 1) # Kansas
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 1328, 2.5), axis = 1) # Oklahoma
submission['Pred'] = submission.apply(lambda x: team_adjustment(x, 1210, 3), axis = 1) # GT

In [15]:
# run same code to compare new altered probabilities to Vegas odds

# merge submission with team slots to look at Round 1 spreads
submission_r1 = pd.merge(submission, team_slots, on = ['TeamID_x', 'TeamID_y'])

# merge vegas odds with submission spreads for round 1
submission_r1 = pd.merge(submission_r1, spread, on = 'ID')

# Calculate absolute difference between my spreads and Vegas'
submission_r1['AbsDiff'] = abs(submission_r1['Spread_x'] - submission_r1['Pred'])

# print submission_r1 ordering by the difference in spread
submission_r1.sort_values('AbsDiff')

,ID,Pred,TeamID_x,TeamID_y,TeamName_x,TeamName_y,Spread_x,AbsDiff
8,2021_1210_1260,5.489495,1210,1260,Georgia Tech,Loyola-Chicago,5.5,0.010505
18,2021_1314_1458,-1.465475,1314,1458,North Carolina,Wisconsin,-1.5,0.034525
15,2021_1281_1328,0.894917,1281,1328,Missouri,Oklahoma,1.0,0.105083
12,2021_1251_1329,7.371238,1251,1329,Liberty,Oklahoma St,7.5,0.128762
29,2021_1332_1433,-5.218921,1332,1433,Oregon,VCU,-5.0,0.218921
11,2021_1104_1233,-16.755543,1104,1233,Alabama,Iona,-16.5,0.255543
21,2021_1326_1331,-16.311216,1326,1331,Ohio St,Oral Roberts,-16.0,0.311216
24,2021_1261_1382,-1.947825,1261,1382,LSU,St Bonaventure,-1.5,0.447825
4,2021_1179_1455,-2.295166,1179,1455,Drake,Wichita St,-1.5,0.795166
1,2021_1155_1353,0.666064,1155,1353,Clemson,Rutgers,1.5,0.833936


In [17]:
# Check play in games and high probability round 2 games
query = 'ID ==\'2021_1211_1328\'' # Gonzaga vs Oklahoma
query = query + ' or ID ==\'2021_1211_1281\'' # Gonzaga vs Missouri
query = query + ' or ID ==\'2021_1104_1268\'' # Alabama vs Maryland
query = query + ' or ID ==\'2021_1326_1439\'' # Ohio State vs Va tech
query = query + ' or ID ==\'2021_1211_1313\'' # Gonzaga v Norfolk St
query = query + ' or ID ==\'2021_1179_1425\'' # Drake vs USC
query = query + ' or ID ==\'2021_1276_1411\'' # Michigan v Texas Southern
query = query + ' or ID ==\'2021_1140_1417\'' # BYU v UCLA
query = query + ' or ID ==\'2021_1140_1400\'' # BYU v Texas
submission.query(query)

,ID,Pred,TeamID_x,TeamID_y
155,2021_1140_1400,3.062062,1140,1400
394,2021_1179_1425,4.496265,1179,1425
924,2021_1104_1268,-4.888835,1104,1268
1012,2021_1211_1281,-13.739563,1211,1281
1185,2021_1211_1313,-26.325695,1211,1313
1372,2021_1211_1328,-15.443746,1211,1328
1832,2021_1276_1411,-22.683251,1276,1411
1872,2021_1140_1417,-8.067756,1140,1417
2106,2021_1326_1439,-4.012169,1326,1439


In [18]:
# make manual adjustments to round 1 win probabilities based on Vegas odds
submission.loc[2106, 'Pred'] = -5 # Ohio State vs Virginia Tech
submission.loc[394, 'Pred'] = 5.5 # Drake v USC
submission.loc[1872, 'Pred'] = -5 # BYU v UCLA
submission.loc[155, 'Pred'] = 4.5 # BYU v Texas

### Kaggle Alterations

Kaggle allows two submission files, so I will use my spreads for the lopsided games in one submission and use Vegas' spreads in the other

In [19]:
query = 'ID ==\'2021_1156_1222\'' # Cleveland St v Houston
query = query + ' or ID ==\'2021_1124_1216\'' # Baylor vs Hartford
query = query + ' or ID ==\'2021_1180_1228\'' # Drexel vs Illinois
query = query + ' or ID ==\'2021_1276_1411\'' # Michigan v Texas Southern
query = query + ' or ID ==\'2021_1211_1313\'' # Gonzaga v Norfolk St
submission.query(query)

,ID,Pred,TeamID_x,TeamID_y
218,2021_1156_1222,16.209415,1156,1222
422,2021_1180_1228,15.705873,1180,1228
689,2021_1124_1216,-21.197699,1124,1216
1185,2021_1211_1313,-26.325695,1211,1313
1832,2021_1276_1411,-22.683251,1276,1411


In [23]:
submission1 = submission.copy()[['ID', 'Pred']]
submission2 = submission.copy()[['ID', 'Pred']]

In [24]:
# make manual adjustments to round 1 win probabilities based on Vegas odds
submission2.loc[218, 'Pred'] = 20  # Cleve St v Houston
submission2.loc[422, 'Pred'] = 22.5  # Drexel v Illinois
submission2.loc[689, 'Pred'] = -25.5  # Baylor v Hartford
submission2.loc[1185, 'Pred'] = -34  # Gonzaga vs Norfolk St
submission2.loc[1832, 'Pred'] = -26  # Michigan v Texas Southern

In [25]:
# write submissions to csv
submission1.to_csv('mydata/mens/submission1_spread.csv', index = False)
submission2.to_csv('mydata/mens/submission2_spread.csv', index = False)